In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.utils import to_categorical

import os
import sys  
sys.path.insert(0, 'G:\\My Drive\\Colab Notebooks\\MWCNN')

from Layers.DTCWT import *
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam, SGD

In [2]:
nb_classes = 10
batch_size = 32
epochs = 40

lr = 1e-4  # learning rate
beta_1 = 0.9         # beta 1 - for adam optimizer
beta_2 = 0.96        # beta 2 - for adam optimizer
epsilon = 1e-7        # epsilon - for adam optimizer

trainFactor = 0.8
input_shape = (28, 28, 1)  # CIFAR-10 60,000 32X32 color

# optimizer = Adam(learning_rate=lr, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon)
optimizer = SGD(lr=lr, momentum=beta_1)

(x_train, y_train), (x_test, y_test) = mnist.load_data()

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
x_train = x_train.astype('float32') / 255.0
x_train = np.expand_dims(x_train, axis=-1)

x_test = x_test.astype('float32') / 255.0
x_test = np.expand_dims(x_test, axis=-1)


In [3]:
# load DWT db2 model
input_shape = (28, 28, 1)  # CIFAR-10 60,000 32X32 color

cplx_input = layers.Input(shape=input_shape)
x = layers.Conv2D(28, (3, 3), padding='same', activation="relu")(cplx_input)
x = DTCWT(2)(x)
x = layers.Conv2D(56, (3, 3), padding='same', activation="relu")(x)
x = DTCWT(2)(x)
x = layers.Dropout(0.5)(x)
x = layers.Flatten()(x)
x = layers.Dense(nb_classes, activation="softmax")(x)
model = Model(cplx_input, x, name="mymodel")
model.summary()



Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "mymodel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 28)        280       
_________________________________________________________________
dtcwt (DTCWT)                (None, 56, 56, 28)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 56)        14168     
_________________________________________________________________
dtcwt_1 (DTCWT)              (None, 112, 112, 56)      0         
_________________________________________________________________
dropout (Dropout)            (None, 112, 112, 56)      0         
____________________________________________

In [4]:
model.compile(loss="categorical_crossentropy",
              optimizer=optimizer, metrics=["accuracy"])

In [ ]:
history = model.fit(x_train, y_train,
                    validation_split=1 - trainFactor,
                    epochs=epochs,
                    batch_size=batch_size,
                    verbose=2,
                    )

Train on 48000 samples, validate on 12000 samples
Epoch 1/40
48000/48000 - 963s - loss: 1.5903 - acc: 0.6983 - val_loss: 0.5588 - val_acc: 0.8718
Epoch 2/40
48000/48000 - 958s - loss: 0.4336 - acc: 0.8842 - val_loss: 0.3273 - val_acc: 0.9091
Epoch 3/40
48000/48000 - 969s - loss: 0.3283 - acc: 0.9066 - val_loss: 0.2823 - val_acc: 0.9193
Epoch 4/40
48000/48000 - 967s - loss: 0.2881 - acc: 0.9178 - val_loss: 0.2518 - val_acc: 0.9259
Epoch 5/40
48000/48000 - 965s - loss: 0.2587 - acc: 0.9257 - val_loss: 0.2314 - val_acc: 0.9342
Epoch 6/40
48000/48000 - 967s - loss: 0.2351 - acc: 0.9331 - val_loss: 0.2099 - val_acc: 0.9413
Epoch 7/40
48000/48000 - 963s - loss: 0.2144 - acc: 0.9393 - val_loss: 0.1921 - val_acc: 0.9467
Epoch 8/40
48000/48000 - 966s - loss: 0.1963 - acc: 0.9439 - val_loss: 0.1815 - val_acc: 0.9491
Epoch 9/40
48000/48000 - 970s - loss: 0.1795 - acc: 0.9488 - val_loss: 0.1632 - val_acc: 0.9555
Epoch 10/40
48000/48000 - 962s - loss: 0.1650 - acc: 0.9535 - val_loss: 0.1515 - val_a

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')

plt.show()